# Simon algorithm(サイモン アルゴリズム)（概要）

Simon algorithm を説明します。

このアルゴリズムでは $n$ ビット入出力を持つ関数 $f_s(x)$ ($s$ は任意の $n$ ビット列)について、以下のどちらかが成り立つものとします。

1. 異なる入力に対し、必ず異なる出力を返す(1対1対応)

2. 入力 $x, x'$ について $x' = x\oplus s$ ならば、$f_s(x) = f_s(x')$. すなわち、2つの入力に対し同じ出力を返す.


このアルゴリズムでは Oracle が上の1. か 2. かを判別します。

具体的な量子回路は次のとおりです。$U_f$ の中身は上記 2\.、$s=1001$ の場合を示しています。  
量子ビット数は $2 n$ です。

<img src="./img/103_img.png" width="50%">

それぞれの状態を確認します。

$$
\begin{align}
\lvert \psi_1\rangle &= \frac{1}{\sqrt{2^n}} \biggl(\otimes^n H\lvert 0\rangle \biggr) \lvert 0\rangle^{\otimes n} \\
&= \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} \lvert x\rangle \lvert 0\rangle^{\otimes n}
\end{align}
$$

次に $\lvert \psi_2 \rangle$ を考えます。

ここでは$f_s(x)$について、次にようなオラクルゲート $U_f$ を用意します。

$$
U_f \lvert x \rangle \lvert 0 \rangle = \lvert x \rangle \lvert f_s(x) \rangle
$$

この $U_f$ を用いると、

$$
\lvert \psi_2 \rangle = \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} \lvert x\rangle \lvert f_s(x)\rangle
$$

よって $\lvert \psi_3 \rangle$ は次のようになります。

$$
\lvert \psi_3 \rangle = \frac{1}{2^n} \sum_{x=0}^{2^n-1}\sum_{y=0}^{2^n-1} (-1)^{x\cdot y} \lvert y\rangle \lvert f_s(x)\rangle
$$

ここで、$f_s(x)$ が次のような場合に $\lvert y \rangle$ の測定結果がどうなるか考えましょう。

1\. 異なる入力に対し、必ず異なる出力を返す(1対1対応)場合.

全ての測定結果が均等な確率で得られます。

2\. 入力 $x, x'$ について $x' = x\oplus s$ ならば、$f_s(x) = f_s(x')$. すなわち、2つの入力に対し同じ出力を返す場合.

1つの状態 $\lvert y \rangle \lvert f_s(x) \rangle = \lvert y \rangle \lvert f_s(x\oplus s) \rangle$ の振幅 $A(y, x)$ に注目します。

$$
A(y, x) = \frac{1}{2^n} \{(-1)^{x\cdot y} + (-1)^{(x\oplus s) \cdot y}\}
$$

式からわかるように、$y\cdot s \equiv 1 \bmod2$ となる $y$ は打ち消し合いにより振幅 $0$ となります。  
よって、$y\cdot s \equiv 0 \bmod2$ となるような $y$ のみが測定されます。

1\. と 2\. のどちらの場合でも、複数回の測定によってこのような $y$ が $n$ 通り(ただし $00...0$ を除く)得られれば、それら全ての $y$ について $y\cdot s' \equiv 0 \bmod2$ となるような $s'$ が決まります。

1\. の場合には $s'$ は完全にランダムです。  
しかし 2\. の場合は、$s' = 0\oplus s'$ のため必ず $f_s(s') = f_s(0)$ となります。

よって、1\. から偶然 $f_s(s') = f_s(0)$ なる $s'$ が得られた場合(確率 $1 / 2^n$)を除き、$s'$ が 1\. と 2\. のどちらから得られたかオラクルゲートを用いて確認できます。  
以上よりオラクルを判別できます。

最後にオラクルゲート $U_f$ の実装を考えます。

1\. の場合は、入力 $x$ に対して出力が1対1対応しさえすれば良いです。  
簡単に、$X$ ゲートをランダムに挿入する回路を考えましょう。

2\. の場合は、少し複雑です。  
まず$CX$ゲートにより次のような状態を作ります。

$$
\lvert \psi_{1a} \rangle = \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} \lvert x\rangle \lvert x\rangle
$$

次に、$s_i=1$ である最も下位のインデックス $i'$ について、$x_{i'} = 0$の場合のみ補助レジスタと $s$ のXORをとります。  
結果として、次のような$\lvert \psi_2 \rangle$ を得ます。

$$
\begin{align}
\lvert \psi_{2} \rangle &= \frac{1}{\sqrt{2^n}} \biggl(\sum_{\{x_{i'}=0\}} \lvert x\rangle \lvert x \oplus s\rangle + \sum_{\{x_{i'}=1\}} \lvert x\rangle \lvert x\rangle \biggr) \\
&= \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} \lvert x\rangle \lvert f_s(x)\rangle
\end{align}
$$

この $f_s(x)$ は2\. を満たすことが計算により確かめられます。

これをblueqatで実装してみましょう。

In [178]:
from blueqat import Circuit
import numpy as np

2種類のオラクルゲート $U_f$ を作用させる関数を用意します。

In [179]:
def oracle_1(c, s):
    _n = len(s)
    for i in range(_n):
        if np.random.rand() > 0.5:
            c.x[i]
    for i in range(_n):
        c.cx[i, i + _n]
        
def oracle_2(c, s):
    _n = len(s)
    flag = 0
    for i, si in enumerate(reversed(s)):
        c.cx[i, i + _n]
        if si == '1' and flag == 0:
            c.x[i]
            for j, sj in enumerate(s):
                if sj == '1':
                    c.cx[i, j + _n]
            c.x[i]
            flag = 1

以下がアルゴリズム本体です。  
最初に、オラクル(2種類のいずれか)と求めたい $s$ を乱数で決めます。  
(以下では上図の量子回路を再現するように値を固定しています)

In [180]:
n = 4
N = np.random.randint(1, 2**n-1)
s = bin(N)[2:].zfill(n)
which_oracle = np.random.rand()

### 上量子回路図の結果を再現するため ###
### この2行を消すとランダムに決まります ###
s = "1001" 
which_oracle = 0
######

c = Circuit(n * 2)
c.h[:n]

if which_oracle > 0.5:
    oracle_1(c, s)
    oracle = "oracle 1"
else:
    oracle_2(c, s)
    oracle = "oracle 2"
    
c.h[:n].m[:n]
res = c.run(shots = 1000)

In [181]:
res

Counter({'10010000': 111,
         '00100000': 120,
         '10110000': 128,
         '11110000': 123,
         '01100000': 137,
         '01000000': 133,
         '00000000': 115,
         '11010000': 133})

サンプリング結果から、'00...0' 以外の結果を $n$ 個抜き出します。  

In [182]:
res_list = list(res.keys())
_res_list = []
for i in res_list:
    if i[:n] != '0'*4:
        _res_list.append(i[:n])
    if len(_res_list) == 4:
        break
            
print(_res_list)

['1001', '0010', '1011', '1111']


抜き出した結果から $s'$ を求めます。  
(ここでは単純に条件に合う $s'$ を総当りで探していますが、線形代数的に効率よく求めることが可能です。)

2\. の場合のオラクルが選択されていれば、得られた $s'$ は $s$ と等しくなるはずです。

In [185]:
for i in range(2**n):
    l = bin(i)[2:].zfill(n)
    flag = 1
    for sampled in _res_list: 
        mod = np.sum(np.array(list(l), dtype = np.int) * np.array(list(sampled), dtype = np.int)) % 2
        if mod:
            flag = 0
            break
    if flag:
        output_s = l

In [186]:
print("s' =", output_s)
print("s =", s)

s' = 1001
s = 1001
